In [1]:
import netCDF4 as nc
import numpy as np
import pandas as pd 
import os
import plotly.express as px
import json
import pycountry
import dash
import dash_core_components as dcc
import dash_html_components as html
import data_proc
import draw

csv_path = './data/GlobalLandTemperaturesByCountry.csv'
json_path = './data/countries.geo.json'
nc_path = './data/Raw_TAVG_EqualArea.nc'
#nc_path = './data/Raw_TAVG_LatLong1.nc'

In [2]:
tbc = data_proc.CSVReader(csv_path).tbc

In [12]:
tw = tbc.query("alpha_3=='CHN'")
tw['alpha_3']='TWN'
tw

,AverageTemperature,country,year,alpha_3
8766,5.013600,China,1820,TWN
8767,6.284167,China,1821,TWN
8768,6.341417,China,1822,TWN
8769,7.024364,China,1823,TWN
8770,NaN,China,1824,TWN
...,...,...,...,...
8955,7.809500,China,2009,TWN
8956,7.548417,China,2010,TWN
8957,7.357083,China,2011,TWN
8958,7.077083,China,2012,TWN


In [17]:
tbc.append(tw).reset_index(drop=True)

,AverageTemperature,country,year,alpha_3
0,18.379571,Afghanistan,1838,AFG
1,NaN,Afghanistan,1839,AFG
2,13.413455,Afghanistan,1840,AFG
3,13.997600,Afghanistan,1841,AFG
4,15.154667,Afghanistan,1842,AFG
...,...,...,...,...
38063,7.809500,China,2009,TWN
38064,7.548417,China,2010,TWN
38065,7.357083,China,2011,TWN
38066,7.077083,China,2012,TWN


In [16]:
tbc

,AverageTemperature,country,year,alpha_3
0,18.379571,Afghanistan,1838,AFG
1,NaN,Afghanistan,1839,AFG
2,13.413455,Afghanistan,1840,AFG
3,13.997600,Afghanistan,1841,AFG
4,15.154667,Afghanistan,1842,AFG
...,...,...,...,...
46805,21.377250,Zimbabwe,2009,ZWE
46806,21.986250,Zimbabwe,2010,ZWE
46807,21.602417,Zimbabwe,2011,ZWE
46808,21.521333,Zimbabwe,2012,ZWE
